In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
from time import sleep
import json

## Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:

**- Наименование вакансии.**

**- Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).**

**- Ссылку на саму вакансию.**

**- Сайт, откуда собрана вакансия.** 

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 

Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.
__________________________________________

Написал функцию, которая, в зависимости от количества переданных в неё аргументов, парсит вакансии с сайтов:

**- HH - HearHunter.ru**

**- SJ - SuperJob.ru**

**- RR - Rabota.ru**

Можно собрать информацию с одного сайта, можно с двух, можно с трёх, как душа пожелает))

Вакансии собираются и сохраняются JSON-файлы, для каждого сайта свой под своим названием. **Информация со страниц собирается с пингом в 5 секунд на страницу** (на всякий случай)

Для примера взял вакансии продавца-консультанта, так как их, во-первых, много у всех, а во-вторых, по ним хорошо видно и разброс зарплат (от и до) и в целом ёмкость сайтов по количеству размещённых вакансий на них. 

Продавцы-консультанты в запросе на каждом сайте названы по своему - на HeadHunter.ru - prodavets-konsultant, на Superjob - prodavec-konsultant.html, на Rabota.ru - specialization_ids=2195. И это единственная информация в коде, которую нужно будет менять для получения списка вакансий по другим специальностям. В остальном код универсален и возвращает структуру данных в таком виде:

**номер вакансии** (счётчик), а в нем: 

- название ('str')

- ссылка ('str')

- сумма зарплаты от ('int')

- сумма зарплаты до ('int')

- сайт ('str')

In [2]:
def job_scraping(*args):
    
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    try:
        for website in list(args):
            # в зависимости от сайта:
            if website == 'HH': 
                params = {'page': 0, 'hhtmFrom': 'vacancy_search_catalog'}
                job_search = 'prodavets-konsultant'
                site_name = 'https://hh.ru/vacancies/'
                url, name_file = f'{site_name}{job_search}', 'HH_vacancies.json'
            if website == 'SJ':
                params = {'page': 1}
                site_name = 'https://www.superjob.ru/'
                job_search = 'prodavec-konsultant.html'
                url, name_file = f'{site_name}vakansii/{job_search}', 'SJ_vacancies.json'
            if website == 'RR':
                params = {'sort': 'relevance', 'specialization_ids': '2195', 'page': 1}
                job_search = 'vacancy/'
                site_name = 'https://www.rabota.ru/'
                url, name_file = f'{site_name}{job_search}', 'RR_vacancies.json'

            session = requests.Session()
            response = session.get(url, headers=headers, params=params)
            json_vacancies, num, start_node, adding_a_path = {}, 0, list(args), site_name

            while len(start_node) > 0:   
                print(f"Обработка страницы № {params['page']}")  
                dom = bs(response.text, 'html.parser')
                if website == 'HH':
                    start_node = dom.find_all('div', {'class': ['vacancy-serp-item-body__main-info']})
                if website == 'SJ':
                    start_node = dom.find_all('div', {'class': '_2lp1U _2J-3z _3B5DQ'})
                if website == 'RR':
                    start_node = dom.find_all('header', {'class': 'vacancy-preview-card__header'})

                for vacancy in start_node:
                    num += 1
                    minimal_salary, maximal_salary, currency_name, currency_name_letter = None, None, None, []
                    if website == 'HH':
                        vacansy_text_and_link = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
                        vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
                        adding_a_path = 'h'
                    if website == 'SJ':
                        vacansy_text_and_link = vacancy.find('a', {'class': 'YrERR'})
                        vacancy_salary = vacancy.find('span', {'class': '_2eYAG'})
                    if website == 'RR':
                        vacansy_text_and_link = vacancy.find('a', {'class': 'vacancy-preview-card__title_border'})
                        vacancy_salary = vacancy.find('div', {'class': 'vacancy-preview-card__salary'}).findChildren(recursive=False)[0]
                    # здесь обрабатываем строчку с зарплатой, превращая цифры в int, а валюту в str 
                    if vacancy_salary:
                        string_salary = vacancy_salary.text
                        for letter in string_salary[::-1]:
                            if letter.isdigit():
                                break
                            currency_name_letter.append(letter)
                        currency_name = ''.join(currency_name_letter[::-1])
                        currency_name = currency_name.replace('\xa0', '')
                        regular_salary = re.findall('(\d+)', string_salary)
                        if len(regular_salary) == 2:
                            if not string_salary.find('до'):
                                maximal_salary = int(regular_salary[0]+regular_salary[1])
                            else:
                                minimal_salary = int(regular_salary[0]+regular_salary[1])
                        elif len(regular_salary) == 0:
                            pass
                        else:
                            minimal_salary = int(regular_salary[0]+regular_salary[1])
                            maximal_salary = int(regular_salary[2]+regular_salary[3])
                    # собираем словари
                    json_vacancies[num] = {
                        'название': vacansy_text_and_link.text.replace('\n', '').strip(),
                        'ссылка': adding_a_path + vacansy_text_and_link['href'][1:],
                        'зарплата от': minimal_salary,
                        'зарплата до': maximal_salary,
                        'валюта': currency_name,
                        'сайт': site_name}

                response = session.get(url, headers=headers, params=params)
                params['page'] += 1
                sleep(5)

            with open(name_file, 'w', encoding='UTF-8') as f:
                json.dump(json_vacancies, f)
                print(f'{name_file} создан! сохранено вакансий {num} шт.')
    except:
        print("эй, переданы не те аргументы, надо эти: 'HH', 'SJ', 'RR'")

запустим все 3 сайта:

In [3]:
job_scraping('HH', 'SJ', 'RR')

Обработка страницы № 0
Обработка страницы № 1
Обработка страницы № 2
Обработка страницы № 3
Обработка страницы № 4
Обработка страницы № 5
Обработка страницы № 6
Обработка страницы № 7
Обработка страницы № 8
Обработка страницы № 9
Обработка страницы № 10
Обработка страницы № 11
Обработка страницы № 12
Обработка страницы № 13
Обработка страницы № 14
Обработка страницы № 15
Обработка страницы № 16
Обработка страницы № 17
Обработка страницы № 18
Обработка страницы № 19
Обработка страницы № 20
Обработка страницы № 21
Обработка страницы № 22
Обработка страницы № 23
Обработка страницы № 24
Обработка страницы № 25
Обработка страницы № 26
Обработка страницы № 27
Обработка страницы № 28
Обработка страницы № 29
Обработка страницы № 30
Обработка страницы № 31
Обработка страницы № 32
Обработка страницы № 33
Обработка страницы № 34
Обработка страницы № 35
Обработка страницы № 36
Обработка страницы № 37
Обработка страницы № 38
Обработка страницы № 39
Обработка страницы № 40
Обработка страницы № 41
HH

### проверим за собой, прочитаем по паре вакансий из каждого файла:

In [4]:
with open('HH_vacancies.json', 'r', encoding='UTF-8') as f:
    HH = json.load(f)
print(f'HeadHunter.ru - количество вакансий в файле: {len(HH)}\n')
pprint(HH['20'])
pprint(HH['999'])

HeadHunter.ru - количество вакансий в файле: 2051

{'валюта': ' руб.',
 'зарплата до': None,
 'зарплата от': 80000,
 'название': 'Продавец-консультант мебели класса люкс',
 'сайт': 'https://hh.ru/vacancies/',
 'ссылка': 'https://hh.ru/vacancy/34259161?query=%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B2%D0%B5%D1%86-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82'}
{'валюта': ' руб.',
 'зарплата до': None,
 'зарплата от': 73000,
 'название': 'Продавец-консультант, продавец (м. Полежаевская, ТЦ "Хорошо")',
 'сайт': 'https://hh.ru/vacancies/',
 'ссылка': 'https://hh.ru/vacancy/67579746?query=%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B2%D0%B5%D1%86-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82'}


In [5]:
with open('SJ_vacancies.json', 'r', encoding='UTF-8') as f:
    SJ = json.load(f)
print(f'SuperJob.ru - количество вакансий в файле: {len(SJ)}\n')
pprint(SJ['20'])
pprint(SJ['199'])

SuperJob.ru - количество вакансий в файле: 288

{'валюта': 'руб.',
 'зарплата до': None,
 'зарплата от': 75000,
 'название': 'Продавец-консультант',
 'сайт': 'https://www.superjob.ru/',
 'ссылка': 'https://www.superjob.ru/vakansii/prodavec-konsultant-44366143.html'}
{'валюта': 'руб.',
 'зарплата до': 40000,
 'зарплата от': None,
 'название': 'Продавец-консультант (отдел Церковная утварь)',
 'сайт': 'https://www.superjob.ru/',
 'ссылка': 'https://www.superjob.ru/vakansii/prodavec-konsultant-31168468.html'}


In [6]:
with open('RR_vacancies.json', 'r', encoding='UTF-8') as f:
    RR = json.load(f)
print(f'Rabota.ru - количество вакансий в файле: {len(RR)}\n')
pprint(RR['20'])
pprint(RR['800'])

Rabota.ru - количество вакансий в файле: 1015

{'валюта': ' руб.',
 'зарплата до': None,
 'зарплата от': 35000,
 'название': 'Продавец-консультант',
 'сайт': 'https://www.rabota.ru/',
 'ссылка': 'https://www.rabota.ru/vacancy/37891969/?search_id=1662658427775jvcwq97zhpr'}
{'валюта': ' руб.',
 'зарплата до': 50000,
 'зарплата от': 40000,
 'название': 'Продавец-консультант/Продавец-кассир (Одинцово)',
 'сайт': 'https://www.rabota.ru/',
 'ссылка': 'https://www.rabota.ru/vacancy/46880178/'}
